In [ ]:
"""
Meta-analysis Data Preparation Script 
====================================================

Input Files:
-----------
- moderator_raw.csv: Study-level moderator variables
- effect_size_raw_results.csv: Effect-size estimates (Hedges' g, SE, CI)

Output File:
-----------
- meta_ready_cleaned_23studies.csv: Final merged, cleaned dataset

"""

import numpy as np
import pandas as pd


# ============================================================================
# STEP 1: LOAD RAW DATA FILES
# ============================================================================
print("=" * 70)
print("STEP 1: Load raw data files")
print("=" * 70)

# Load CSV files as strings to preserve original formatting and coding
# dtype=str prevents automatic type conversion that might lose information
# encoding="utf-8-sig" handles BOM (byte order mark) in Excel-exported CSVs
moderator_raw = pd.read_csv("moderator_raw.csv", dtype=str, encoding="utf-8-sig")
effect_size_raw = pd.read_csv("effect_size_raw_results.csv", dtype=str, encoding="utf-8-sig")

print(f"Moderator file: {moderator_raw.shape[0]} rows × {moderator_raw.shape[1]} columns")
print(f"Effect-size file: {effect_size_raw.shape[0]} rows × {effect_size_raw.shape[1]} columns")

# Remove leading/trailing whitespace from all string cells
# This prevents merge failures due to inconsistent spacing
moderator_raw = moderator_raw.applymap(lambda x: x.strip() if isinstance(x, str) else x)
effect_size_raw = effect_size_raw.applymap(lambda x: x.strip() if isinstance(x, str) else x)


# ============================================================================
# STEP 2: STANDARDIZE 'INDEX' COLUMN AND FILTER TO COMMON INDICES
# ============================================================================
print("\n" + "=" * 70)
print("STEP 2: Standardize 'index' column and filter to common indices")
print("=" * 70)

# Clean the 'index' column in both tables
# Convert to string and remove any whitespace to ensure consistent matching
moderator_raw["index"] = moderator_raw["index"].astype(str).str.strip()
effect_size_raw["index"] = effect_size_raw["index"].astype(str).str.strip()

# Identify indices that exist in BOTH tables
# Using set intersection to find common indices
indices_moderator = set(moderator_raw["index"])
indices_effect = set(effect_size_raw["index"])
common_indices = indices_moderator & indices_effect  # Intersection

print(f"Indices in moderator file: {len(indices_moderator)}")
print(f"Indices in effect-size file: {len(indices_effect)}")
print(f"Common indices (in both): {len(common_indices)}")

# Identify indices that appear in only one file
# These will be excluded from the final dataset
only_moderator = sorted(indices_moderator - indices_effect)
only_effect = sorted(indices_effect - indices_moderator)

if only_moderator:
    print(f"\n⚠️  Indices only in moderator file (will be excluded): {only_moderator}")
if only_effect:
    print(f"⚠️  Indices only in effect-size file (will be excluded): {only_effect}")

# Filter BOTH tables to retain only common indices
# This ensures a 1:1 merge without missing data on either side
moderator_raw = moderator_raw[moderator_raw["index"].isin(common_indices)].copy()
effect_size_raw = effect_size_raw[effect_size_raw["index"].isin(common_indices)].copy()

print(f"\nAfter filtering:")
print(f"Moderator file: {moderator_raw.shape[0]} rows")
print(f"Effect-size file: {effect_size_raw.shape[0]} rows")


# ============================================================================
# STEP 3: NORMALIZE MISSING-VALUE CODES TO NaN
# ============================================================================
print("\n" + "=" * 70)
print("STEP 3: Normalize missing-value codes to NaN")
print("=" * 70)

# Define all representations of missing data used in the raw files
# These will be standardized to numpy NaN for consistent handling
missing_codes = [
    "NG",      # Not Reported
    "N/R",     # Not Reported (alternative)
    "n/a",     # Not Applicable
    "N/A",     # Not Applicable (capitalized)
    "NA",      # Not Available
    "NaN",     # Already NaN as string
    "",        # Empty string
    "-",       # Dash used as placeholder
    "Mixed",   # Mixed/heterogeneous (cannot be analyzed as single category)
    "mixed",   # Mixed (lowercase)
    "MIXED"    # Mixed (uppercase)
]

# Replace all missing codes with numpy NaN across both dataframes
moderator_raw = moderator_raw.replace(missing_codes, np.nan)
effect_size_raw = effect_size_raw.replace(missing_codes, np.nan)

print("Missing codes normalized: NG, N/A, Mixed, etc. → NaN")


# ============================================================================
# STEP 4: STANDARDIZE YES/NO VALUES
# ============================================================================
print("\n" + "=" * 70)
print("STEP 4: Standardize Yes/No values")
print("=" * 70)

# Map all variations of Yes/No to standard capitalized format
# This ensures consistency for categorical moderator analysis
yes_no_map = {
    # Yes variations
    "yes": "Yes",
    "YES": "Yes",
    "y": "Yes",
    "Y": "Yes",
    
    # No variations
    "no": "No",
    "NO": "No",
    "n": "No",
    "N": "No"
}

moderator_raw = moderator_raw.replace(yes_no_map)
effect_size_raw = effect_size_raw.replace(yes_no_map)

print("Yes/No values standardized (yes/no/y/n → Yes/No)")


# ============================================================================
# STEP 5: STANDARDIZE CATEGORICAL LABELS
# ============================================================================
print("\n" + "=" * 70)
print("STEP 5: Standardize categorical labels")
print("=" * 70)

# --- Standardize Age_Group ---
# Convert to consistent capitalization: Adult, Adolescent
if "Age_Group" in moderator_raw.columns:
    moderator_raw["Age_Group"] = moderator_raw["Age_Group"].replace({
        "adult": "Adult",
        "adolescent": "Adolescent",
        "adolescent ": "Adolescent"  # Handle trailing space
    })
    print("Age_Group standardized: adult → Adult, adolescent → Adolescent")

# --- Standardize Proficiency_Level ---
# Convert to consistent capitalization
# Mixed/compound levels (e.g., "Intermediate_Advanced") → NaN
# because they cannot be analyzed as a single proficiency category
if "Proficiency_Level" in moderator_raw.columns:
    moderator_raw["Proficiency_Level"] = moderator_raw["Proficiency_Level"].replace({
        "intermediate": "Intermediate",
        "Intermediate_Advanced": np.nan,  # Compound level → missing
        "mixed": np.nan,
        "Mixed": np.nan
    })
    print("Proficiency_Level standardized: intermediate → Intermediate")
    print("  (Mixed/compound levels set to NaN)")


# ============================================================================
# STEP 6: TRANSFORM Gender_Ratio_FM TO PROPORTION OF FEMALES
# ============================================================================
print("\n" + "=" * 70)
print("STEP 6: Transform Gender_Ratio_FM to proportion of females")
print("=" * 70)

if "Gender_Ratio_FM" in moderator_raw.columns:
    # Store original column position to maintain column order
    gender_col_position = moderator_raw.columns.get_loc("Gender_Ratio_FM")
    
    print("Original Gender_Ratio_FM values (sample):")
    print(moderator_raw["Gender_Ratio_FM"].head(12).tolist())
    
    # --- Extract female and male counts ---
    # Pattern: "numberF/numberM" (e.g., "20F/12M")
    # Regex explanation:
    #   ^\s*        : Start of string, optional whitespace
    #   (\d+)       : Capture group 1 - one or more digits (female count)
    #   \s*[Ff]\s*  : Optional spaces + F or f + optional spaces
    #   /           : Literal forward slash
    #   \s*         : Optional whitespace
    #   (\d+)       : Capture group 2 - one or more digits (male count)
    #   \s*[Mm]\s*  : Optional spaces + M or m + optional spaces
    #   $           : End of string
    gender_parts = moderator_raw["Gender_Ratio_FM"].str.extract(
        r"^\s*(\d+)\s*[Ff]\s*/\s*(\d+)\s*[Mm]\s*$"
    )
    
    # Convert extracted strings to numeric
    n_female = pd.to_numeric(gender_parts[0], errors="coerce")
    n_male = pd.to_numeric(gender_parts[1], errors="coerce")
    
    # Calculate total participants and proportion female
    n_total = n_female + n_male
    proportion_female = n_female / n_total
    
    # Round to 2 decimal places for interpretability
    # Example: 20F/12M → 20/32 = 0.625 → 0.62
    proportion_female = proportion_female.round(2)
    
    # Replace original column with proportion values
    moderator_raw["Gender_Ratio_FM"] = proportion_female
    
    # Ensure column stays in original position (not moved to end)
    cols = moderator_raw.columns.tolist()
    if cols[gender_col_position] != "Gender_Ratio_FM":
        cols.remove("Gender_Ratio_FM")
        cols.insert(gender_col_position, "Gender_Ratio_FM")
        moderator_raw = moderator_raw[cols]
    
    print("\nTransformed to proportion (female/total, 2 decimals):")
    print(moderator_raw["Gender_Ratio_FM"].head(12).tolist())
    print(f"✓ Gender_Ratio_FM now represents proportion of females (0-1 scale)")


# ============================================================================
# STEP 7: TRANSFORM Training_TotalMinute TO NUMERIC
# ============================================================================
print("\n" + "=" * 70)
print("STEP 7: Transform Training_TotalMinute to numeric")
print("=" * 70)

if "Training_TotalMinute" in moderator_raw.columns:
    print("Original Training_TotalMinute values (sample):")
    print(moderator_raw["Training_TotalMinute"].head(10).tolist())
    
    # Get raw values as strings
    minutes_str = moderator_raw["Training_TotalMinute"].astype(str).str.strip()
    
    # Remove internal spaces (e.g., "13 * 90" → "13*90")
    minutes_str = minutes_str.str.replace(" ", "", regex=False)
    
    # Initialize result series with NaN values
    minutes_numeric = pd.Series([np.nan] * len(minutes_str), index=moderator_raw.index)
    
    # --- Process each value individually ---
    for idx, val in minutes_str.items():
        # Case 1: Missing or empty value → keep as NaN
        if pd.isna(val) or val == "nan" or val == "":
            minutes_numeric[idx] = np.nan
        
        # Case 2: Multiplication expression (e.g., "13*90")
        # This represents: weeks * minutes_per_week = total_minutes
        elif "*" in val:
            try:
                parts = val.split("*")
                if len(parts) == 2:
                    # Multiply the two numbers
                    result = float(parts[0]) * float(parts[1])
                    minutes_numeric[idx] = round(result, 2)
                else:
                    # Invalid format (more than one * symbol)
                    minutes_numeric[idx] = np.nan
            except:
                # Parsing error → NaN
                minutes_numeric[idx] = np.nan
        
        # Case 3: Direct numeric value (e.g., "240", "1200")
        else:
            try:
                minutes_numeric[idx] = round(float(val), 2)
            except:
                # Cannot convert to number → NaN
                minutes_numeric[idx] = np.nan
    
    # Replace original column with numeric values
    moderator_raw["Training_TotalMinute"] = minutes_numeric
    
    print("\nTransformed to numeric minutes:")
    print(moderator_raw["Training_TotalMinute"].head(10).tolist())
    print("✓ Training_TotalMinute: expressions evaluated (e.g., '13*90' → 1170.0)")

# --- Also convert Training_TotalWeeks to numeric ---
if "Training_TotalWeeks" in moderator_raw.columns:
    moderator_raw["Training_TotalWeeks"] = pd.to_numeric(
        moderator_raw["Training_TotalWeeks"], errors="coerce"
    )
    print("✓ Training_TotalWeeks converted to numeric")


# ============================================================================
# STEP 7.5: CREATE Treatment_Duration CATEGORICAL VARIABLE
# ============================================================================
print("\n" + "=" * 70)
print("STEP 7.5: Create Treatment_Duration categorical variable")
print("=" * 70)

if "Training_TotalWeeks" in moderator_raw.columns:
    print("Original Training_TotalWeeks values (sample):")
    print(moderator_raw["Training_TotalWeeks"].head(10).tolist())
    
    # Create Treatment_Duration based on Training_TotalWeeks
    # Classification:
    #   1-4 weeks   → Short
    #   5-8 weeks   → Medium
    #   9+ weeks    → Long
    #   Missing/NaN → NaN
    
    def categorize_duration(weeks):
        """
        Categorize training duration into Short/Medium/Long
        
        Parameters:
        -----------
        weeks : float or NaN
            Number of training weeks
        
        Returns:
        --------
        str or NaN
            'Short' (1-4 weeks), 'Medium' (5-8 weeks), or 'Long' (9+ weeks)
        """
        if pd.isna(weeks):
            return np.nan
        elif weeks <= 4:
            return "Short"
        elif weeks <= 8:
            return "Medium"
        else:  # weeks >= 9
            return "Long"
    
    # Apply categorization
    moderator_raw["Treatment_Duration"] = moderator_raw["Training_TotalWeeks"].apply(categorize_duration)
    
    # Insert Treatment_Duration column right after Training_TotalWeeks
    # to keep related variables together
    weeks_col_position = moderator_raw.columns.get_loc("Training_TotalWeeks")
    cols = moderator_raw.columns.tolist()
    
    # Remove Treatment_Duration from its current position
    cols.remove("Treatment_Duration")
    # Insert it right after Training_TotalWeeks
    cols.insert(weeks_col_position + 1, "Treatment_Duration")
    moderator_raw = moderator_raw[cols]
    
    print("\nTreatment_Duration created:")
    print("  • Short (1-4 weeks)")
    print("  • Medium (5-8 weeks)")
    print("  • Long (≥9 weeks)")
    
    # Show distribution
    duration_counts = moderator_raw["Treatment_Duration"].value_counts(dropna=False)
    print("\nDistribution:")
    for category in ["Short", "Medium", "Long"]:
        count = duration_counts.get(category, 0)
        print(f"  {category:8s}: {count:2d} studies")
    
    missing_count = moderator_raw["Treatment_Duration"].isna().sum()
    if missing_count > 0:
        print(f"  Missing : {missing_count:2d} studies")
    
    print("\n✓ Treatment_Duration variable created successfully")


# ============================================================================
# STEP 8: PREPARE EFFECT-SIZE DATA FOR MERGING
# ============================================================================
print("\n" + "=" * 70)
print("STEP 8: Prepare effect-size data for merging")
print("=" * 70)

# Select essential effect-size columns
# Keep only the columns needed for meta-analysis
effect_clean = effect_size_raw[[
    "index",        # Merge key
    "Study_ID",     # Study identifier
    "Effect_ID",    # Effect-size identifier
    "Hedges_g",     # Effect-size estimate (bias-corrected)
    "SE",           # Standard error
    "Variance",     # Variance of effect size
    "CI_Lower",     # 95% CI lower bound
    "CI_Upper"      # 95% CI upper bound
]].copy()

# Rename Study_ID and Effect_ID to avoid naming conflicts during merge
# These will be used for validation after merging
effect_clean = effect_clean.rename(columns={
    "Study_ID": "Study_ID_effect",
    "Effect_ID": "Effect_ID_effect"
})

print(f"Effect-size data prepared: {effect_clean.shape[0]} rows")


# ============================================================================
# STEP 9: MERGE MODERATOR AND EFFECT-SIZE DATA
# ============================================================================
print("\n" + "=" * 70)
print("STEP 9: Merge moderator and effect-size data")
print("=" * 70)

# Merge on 'index' column
# how="inner": Keep only rows present in BOTH tables
# validate="1:1": Ensure one-to-one relationship (no duplicate indices)
merged_df = pd.merge(
    moderator_raw,
    effect_clean,
    on="index",
    how="inner",      # Inner join: only common indices
    validate="1:1"    # Enforce 1:1 relationship
)

print(f"Merged dataset: {merged_df.shape[0]} rows × {merged_df.shape[1]} columns")


# ============================================================================
# STEP 10: VERIFY DATA INTEGRITY
# ============================================================================
print("\n" + "=" * 70)
print("STEP 10: Verify data integrity")
print("=" * 70)

# --- Check Study_ID consistency ---
# Compare Study_ID from moderator table vs effect-size table
study_mismatch = merged_df[
    merged_df["Study_ID"].astype(str) != merged_df["Study_ID_effect"].astype(str)
]

if len(study_mismatch) > 0:
    print(f"⚠️  Study_ID mismatches found: {len(study_mismatch)}")
    print(study_mismatch[["index", "Study_ID", "Study_ID_effect"]])
else:
    print("✓ All Study_ID values match between moderator and effect-size data")

# --- Check Effect_ID consistency ---
# Compare Effect_ID from moderator table vs effect-size table
effect_mismatch = merged_df[
    merged_df["Effect_ID"].astype(str) != merged_df["Effect_ID_effect"].astype(str)
]

if len(effect_mismatch) > 0:
    print(f"⚠️  Effect_ID mismatches found: {len(effect_mismatch)}")
    print(effect_mismatch[["index", "Effect_ID", "Effect_ID_effect"]])
else:
    print("✓ All Effect_ID values match between moderator and effect-size data")

# Remove duplicate ID columns
# Keep the original columns from the moderator table
merged_df = merged_df.drop(columns=["Study_ID_effect", "Effect_ID_effect"])


# ============================================================================
# STEP 11: EXPORT FINAL DATASET
# ============================================================================
print("\n" + "=" * 70)
print("STEP 11: Export final dataset")
print("=" * 70)

# Save final cleaned dataset to CSV
# index=False: Don't write row numbers
# encoding="utf-8-sig": Include BOM for Excel compatibility
merged_df.to_csv("Meta_ready_cleaned.csv", index=False, encoding="utf-8-sig")

print(f"✅ Final dataset exported: meta_ready_cleaned.csv")
print(f"   Total rows: {merged_df.shape[0]}")
print(f"   Total columns: {merged_df.shape[1]}")
print(f"\nKey transformations:")
print(f"   • Gender_Ratio_FM → proportion of females (0-1, 2 decimals)")
print(f"   • Training_TotalMinute → numeric minutes (expressions evaluated)")
print(f"   • Only studies present in BOTH input files are included")
print(f"\n{'=' * 70}")
print("Data preparation complete. Ready for meta-analysis.")
print("=" * 70)

STEP 1: Load raw data files
Moderator file: 28 rows × 24 columns
Effect-size file: 29 rows × 32 columns

STEP 2: Standardize 'index' column and filter to common indices
Indices in moderator file: 28
Indices in effect-size file: 29
Common indices (in both): 28
⚠️  Indices only in effect-size file (will be excluded): ['259']

After filtering:
Moderator file: 28 rows
Effect-size file: 28 rows

STEP 3: Normalize missing-value codes to NaN
Missing codes normalized: NG, N/A, Mixed, etc. → NaN

STEP 4: Standardize Yes/No values
Yes/No values standardized (yes/no/y/n → Yes/No)

STEP 5: Standardize categorical labels
Age_Group standardized: adult → Adult, adolescent → Adolescent
Proficiency_Level standardized: intermediate → Intermediate
  (Mixed/compound levels set to NaN)

STEP 6: Transform Gender_Ratio_FM to proportion of females
Original Gender_Ratio_FM values (sample):
[nan, '30F/0M', nan, nan, nan, nan, nan, nan, nan, '74F/19M', '74F/19M', '25F/24M']

Transformed to proportion (female/tot

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_16064\3859815534.py:53: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  moderator_raw = moderator_raw.applymap(lambda x: x.strip() if isinstance(x, str) else x)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_16064\3859815534.py:54: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  effect_size_raw = effect_size_raw.applymap(lambda x: x.strip() if isinstance(x, str) else x)
